# Self-Driving Car Simulation

<strong>Autonomous Driving</strong> involves using sensors and algorithms to navigate a vehicle without human intervention. In simulations, reinforcement learning algorithms are often used.

<h4>Key Steps:</h4>

1. Simulation Environment: Set up an environment (e.g., CARLA, OpenAI Gym) to simulate driving.
2. Model Training: Use reinforcement learning to train a model that can make driving decisions.
3. Evaluation: Test the model in the simulation and improve its performance iteratively.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import gym

# Set up the simulation environment
env = gym.make('CarRacing-v0')

# Model architecture
def build_model(input_shape, action_space):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(action_space, activation='linear')
    ])
    return model

# Initialize model
input_shape = (96, 96, 3)  # Example input shape
action_space = env.action_space.shape[0]
model = build_model(input_shape, action_space)
model.compile(optimizer='adam', loss='mse')

# Training loop
for episode in range(1000):
    state = env.reset()
    done = False
    while not done:
        state = np.array(state)
        state = state.reshape(1, 96, 96, 3)  # Reshape for model input
        action = model.predict(state)
        next_state, reward, done, _ = env.step(action[0])  # env.step expects the action to be in a specific format
        next_state = np.array(next_state)
        next_state = next_state.reshape(1, 96, 96, 3)
        
        # Update model with new experience
        target = reward + 0.95 * np.max(model.predict(next_state))  # Q-learning update
        target_f = model.predict(state)
        target_f[0][np.argmax(action)] = target
        model.fit(state, target_f, epochs=1, verbose=0)
        
        state = next_state

# Visualize performance
state = env.reset()
done = False
while not done:
    state = np.array(state)
    state = state.reshape(1, 96, 96, 3)
    action = model.predict(state)
    state, reward, done, _ = env.step(action[0])
    env.render()
env.close()
